<a href="https://colab.research.google.com/github/AndreaRossiUniVR/Massive_Dataset_Extraction_FIGH/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Download REPO**

In [ ]:
!git clone https://github.com/AndreaRossiUniVR/Massive_Dataset_Extraction_FIGH.git

Cloning into 'Massive_Dataset_Extraction_FIGH'...
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 39 (delta 13), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (39/39), 7.32 MiB | 6.46 MiB/s, done.


**Set Up the Env**

In [ ]:
!pip install PyPDF2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 9.1 MB/s eta 0:00:00


In [ ]:
import os

os.chdir('./Massive_Dataset_Extraction_FIGH')

*Download DATASET*


In [ ]:
!python download.py

Downloaded and saved Dataset_FIGH/50678.pdf
Downloaded and saved Dataset_FIGH/50679.pdf
Downloaded and saved Dataset_FIGH/50680.pdf
Downloaded and saved Dataset_FIGH/50681.pdf
Downloaded and saved Dataset_FIGH/50682.pdf
Downloaded and saved Dataset_FIGH/50683.pdf
Downloaded and saved Dataset_FIGH/50684.pdf
Downloaded and saved Dataset_FIGH/50685.pdf
Downloaded and saved Dataset_FIGH/50686.pdf
Downloaded and saved Dataset_FIGH/50687.pdf
Downloaded and saved Dataset_FIGH/50688.pdf
Downloaded and saved Dataset_FIGH/50689.pdf
Downloaded and saved Dataset_FIGH/50690.pdf
Downloaded and saved Dataset_FIGH/50691.pdf
Downloaded and saved Dataset_FIGH/50692.pdf
Downloaded and saved Dataset_FIGH/50693.pdf
Downloaded and saved Dataset_FIGH/50694.pdf
Downloaded and saved Dataset_FIGH/50695.pdf
Downloaded and saved Dataset_FIGH/50696.pdf
Downloaded and saved Dataset_FIGH/50697.pdf
Downloaded and saved Dataset_FIGH/50698.pdf
Downloaded and saved Dataset_FIGH/50699.pdf
Downloaded and saved Dataset_FIG

*Create DB*

In [ ]:
import sqlite3

def setup_db():
    # Connect to the SQLite database (or create it if it doesn't exist)
    conn = sqlite3.connect('fighDB.db')

    # Create a cursor object
    c = conn.cursor()

    # Create table for Games
    c.execute('''
    CREATE TABLE Games (
        id INTEGER PRIMARY KEY,
        home_team_id TEXT,
        away_team_id TEXT,
        home_goals INTEGER,
        away_goals INTEGER,
        category TEXT,
        date TEXT
    )
''')

    # Create table for Teams
    c.execute('''
        CREATE TABLE Teams (
            id TEXT PRIMARY KEY,
            team_name TEXT,
            goals INTEGER,
            goals_against INTEGER,
            category TEXT
        )
    ''')

    # Create table for Players
    c.execute('''
    CREATE TABLE Players (
        id TEXT,
        team_id TEXT,
        game_id INTEGER,
        number INTEGER,
        name TEXT,
        goals INTEGER,
        FOREIGN KEY (team_id) REFERENCES Teams (id),
        FOREIGN KEY (game_id) REFERENCES Games (id),
        PRIMARY KEY (id, game_id)
      )
  ''')

    # Commit the changes and close the connection
    conn.commit()
    conn.close()

if __name__ == "__main__":
    setup_db()


*Data Cleaning + DB INSERT*

In [ ]:
import os
import subprocess

# Folder containing the PDF files
pdf_folder = "Dataset_FIGH"

# Iterate through all files in the folder
for file in os.listdir(pdf_folder):
    # Check if the file is a PDF file
    if file.lower().endswith(".pdf"):
        # Construct the file path
        file_path = os.path.join(pdf_folder, file)
        
        # Call the read.py script with the PDF file as an argument
        subprocess.run(["python", "read.py", file_path])


commit
[['Dataset_FIGH/48933.pdf', '48933', '07/01/2023', 'SERIE A1 FEMMINILE', ''], ['CASALGRANDE PADANA', 24, 35, [{'number': '1', 'name': 'Bonacini, Valentina', 'goals': '0'}, {'number': '2', 'name': 'Rondoni, Elisa', 'goals': '0'}, {'number': '3', 'name': 'Mangone, Asia Kristel', 'goals': '3'}, {'number': '5', 'name': 'Giombetti, Nicole', 'goals': '2'}, {'number': '12', 'name': 'Bordon, Nadia Ayelen', 'goals': '0'}, {'number': '14', 'name': 'Artoni, Simona', 'goals': '0'}, {'number': '15', 'name': 'Rondoni, Chiara', 'goals': '0'}, {'number': '16', 'name': 'Mutti, Caterina Maria', 'goals': '0'}, {'number': '18', 'name': 'Artoni, Alessia', 'goals': '1'}, {'number': '19', 'name': 'Orlandi, Marianna', 'goals': '4'}, {'number': '20', 'name': 'Baroni, Emma', 'goals': '0'}, {'number': '21', 'name': 'Giovannini, Matilde', 'goals': '0'}, {'number': '26', 'name': 'Mattioli, Giulia', 'goals': '1'}, {'number': '33', 'name': 'Lusetti, Gaia', 'goals': '1'}], ''], ['JOMI SALERNO', 35, 24, [{'numb

**DEMO**

*Dislplaing Player's advanced stats*


In [ ]:
import sqlite3
import ipywidgets as widgets
import matplotlib.pyplot as plt

# Connect to the SQLite database
conn = sqlite3.connect('fighDB.db')
c = conn.cursor()

# Get all unique categories
c.execute("SELECT DISTINCT category FROM Teams")
categories = [category[0] for category in c.fetchall()]

# Create the category dropdown menu
category_dropdown = widgets.Dropdown(options=categories, description='Category:')
display(category_dropdown)

# Create the team dropdown menu (empty for now)
team_dropdown = widgets.Dropdown(description='Team:')
display(team_dropdown)

# Create the player dropdown menu (empty for now)
player_dropdown = widgets.Dropdown(description='Player:')
display(player_dropdown)

# Create a button to confirm the category selection
category_button = widgets.Button(description="Confirm Category")
display(category_button)

# Function to update the team dropdown menu once a category is selected and confirmed
def update_teams(button):
    c.execute("SELECT team_name FROM Teams WHERE category=?", (category_dropdown.value,))
    teams = [team[0] for team in c.fetchall()]
    team_dropdown.options = teams

# Set the function to be called when the category button is clicked
category_button.on_click(update_teams)

# Create a button to confirm the team selection
team_button = widgets.Button(description="Confirm Team")
display(team_button)

# Function to update the player dropdown menu once a team is selected and confirmed
def update_players(button):
    team_id = (team_dropdown.value + category_dropdown.value).replace(" ", "")
    c.execute("SELECT name FROM Players WHERE team_id=?", (team_id,))
    players = [player[0] for player in c.fetchall()]
    player_dropdown.options = players

# Set the function to be called when the team button is clicked
team_button.on_click(update_players)

# Function to display the player stats
def display_player_stats(button):
    team_id = (team_dropdown.value + category_dropdown.value).replace(" ", "")
    player_name = player_dropdown.value

    c.execute('''
        SELECT game_id, goals
        FROM Players
        WHERE team_id=? AND name=?
    ''', (team_id, player_name))
    goals_per_game = c.fetchall()

    c.execute('''
        SELECT p.game_id, p.goals
        FROM Players p
        INNER JOIN Games g ON p.game_id = g.id
        WHERE p.team_id=? AND p.name=? AND ((g.home_team_id=? AND g.home_goals > g.away_goals) OR (g.away_team_id=? AND g.away_goals > g.home_goals))
    ''', (team_id, player_name, team_dropdown.value, team_dropdown.value))
    goals_per_game_wins = c.fetchall()

    c.execute('''
        SELECT p.game_id, p.goals
        FROM Players p
        INNER JOIN Games g ON p.game_id = g.id
        WHERE p.team_id=? AND p.name=? AND ((g.home_team_id=? AND g.home_goals < g.away_goals) OR (g.away_team_id=? AND g.away_goals < g.home_goals))
    ''', (team_id, player_name, team_dropdown.value, team_dropdown.value))
    goals_per_game_losses = c.fetchall()

    total_goals = sum(goal[1] for goal in goals_per_game)
    average_goals = total_goals / len(goals_per_game)

    total_goals_wins = sum(goal[1] for goal in goals_per_game_wins)
    average_goals_wins = total_goals_wins / len(goals_per_game_wins) if goals_per_game_wins else 0

    total_goals_losses = sum(goal[1] for goal in goals_per_game_losses)
    average_goals_losses = total_goals_losses / len(goals_per_game_losses) if goals_per_game_losses else 0

    colors = ['green', 'red']
    plt.bar(['Total', 'Wins', 'Losses'], [average_goals, average_goals_wins, average_goals_losses], color=colors)
    plt.title(f'Average Goals Per Game for {player_name}')
    plt.ylabel('Goals')
    plt.show()

# Create a button to confirm the player selection
player_button = widgets.Button(description="Confirm Player")
display(player_button)

# Set the function to be called when the player button is clicked
player_button.on_click(display_player_stats)


Dropdown(description='Category:', options=('SERIE A1 FEMMINILE', 'SERIE  A2 FEMMINILE', 'SERIE A GOLD MASCHILE…

Dropdown(description='Team:', options=(), value=None)

Dropdown(description='Player:', options=(), value=None)

Button(description='Confirm Category', style=ButtonStyle())

Button(description='Confirm Team', style=ButtonStyle())

Button(description='Confirm Player', style=ButtonStyle())

*Goals Distribution*

In [ ]:
import sqlite3
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets

# Connect to the SQLite database
conn = sqlite3.connect('fighDB.db')
c = conn.cursor()

# Get all unique categories
c.execute("SELECT DISTINCT category FROM Teams")
categories = [category[0] for category in c.fetchall()]
categories.append('ALL CATEGORIES')

# Create the category dropdown menu
category_dropdown = widgets.Dropdown(options=categories, description='Category:')
display(category_dropdown)

# Create a button to confirm the category selection
category_button = widgets.Button(description="Confirm Category")
display(category_button)

def load_data(db_path, category=None):
    # Connect to SQLite database
    conn = sqlite3.connect(db_path)

    if category is None:
        # Convert SQLite table to a pandas dataframe
        df_players = pd.read_sql_query("SELECT * from Players", conn)
    else:
        # Convert SQLite tables to pandas dataframes
        df_players = pd.read_sql_query("SELECT * from Players", conn)
        df_teams = pd.read_sql_query("SELECT id, category from Teams", conn)

        # Merge player and team dataframes on team_id
        df_players = pd.merge(df_players, df_teams, left_on='team_id', right_on='id', suffixes=('_player', '_team'))
        
        df_players = df_players[df_players['category'] == category]

    # Filter out players with 0 goals
    df_players = df_players[df_players['goals'] != 0]

    return df_players, conn

def visualize_distribution(df_players, id_column, category=None):
    # Group players by the total number of goals and count the number of players in each group
    goals_distribution = df_players.groupby(id_column)['goals'].sum().value_counts().reset_index()
    
    goals_distribution.columns = ['Total Goals', 'Number of Players']

    # Sort the dataframe by the total number of goals
    goals_distribution = goals_distribution.sort_values('Total Goals')

    # Plot the distribution of goals
    plt.figure(figsize=(12, 6))
    ax = sns.barplot(data=goals_distribution, x='Total Goals', y='Number of Players', color='blue', alpha=0.8, saturation=0.8, errorbar=None, width=0.9)

    if category is None:
        plt.title('Distribution of Goals')
    else:
        plt.title(f'Distribution of Goals for {category} category')

    plt.xlabel('Total Goals')
    plt.ylabel('Number of Players')

    # Rotate x-axis tick labels
    plt.xticks(rotation=90, fontsize=6)  # Set the font size of x-axis tick labels

    # Add a reference line for 1/x distribution
    max_goals_histogram = goals_distribution['Total Goals'].iloc[-1]  # Maximum number of goals in the histogram
    max_players = goals_distribution['Number of Players'].max()  # Maximum number of players for a certain goal count
    x = np.arange(1, max_goals_histogram + 1)  # X values for the 1/x distribution
    a = 0.005  # This can be any non-zero positive value
    b = 0.9  # This controls the steepness of the curve; values less than 1 make the curve less steep
    y = a / (x + b)**b * max_players  # 1/x distribution with a steepness parameter
    y = y / y.max() * max_players  # Scale the 1/x distribution so that its peak matches the peak of the histogram

    # Limit the x-values and y-values to match the last bar of the histogram
    x = x[:len(goals_distribution)]
    y = y[:len(goals_distribution)]
    
    plt.plot(x, y, color='red', linestyle='--', linewidth=2)


    plt.show()

    # Print the text version of the histogram
    print("Text Version of Histogram:")
    for index, row in goals_distribution.iterrows():
        print(f"{row['Total Goals']} goals: {row['Number of Players']:,} players")

def execute_visualization(button):
    db_path = 'fighDB.db'
    if category_dropdown.value == 'ALL CATEGORIES':
        df_players, conn = load_data(db_path)
        visualize_distribution(df_players, 'id')
    else:
        df_players, conn = load_data(db_path, category_dropdown.value)
        visualize_distribution(df_players, 'id_player', category_dropdown.value)

category_button.on_click(execute_visualization)


Dropdown(description='Category:', options=('SERIE A1 FEMMINILE', 'SERIE  A2 FEMMINILE', 'SERIE A GOLD MASCHILE…

Button(description='Confirm Category', style=ButtonStyle())